In [1]:
import pandas as pd
from pathlib import Path
import os
mids_dir = Path("D:\\MIDS-W207")
data = mids_dir/"datasets/soccertrack"
project = mids_dir/"MIDS-W207-Spring24-Soccer-Detection"
analysis = project/"analysis"

# Author: Timothy Majidzadeh
# Date Created: March 7, 2024
# Date Updated: March 12, 2024
# Description: Develop the baseline YOLOv5 model data.
# Notes: [v1] Created program.
# Inputs: Frame-by-frame image data & labels.
# Outputs: The data prepared for use in YOLOv5 using Ultralytics.

In [2]:
top_view_labels = pd.read_pickle(data/"labels/top_view_labels_stacked/top_view_labels.pkl")

In [3]:

def format_labels_to_ultralytics(
    input_df, output_dir=data/"labels", input_type='top_view', 
    input_height=2160, input_width=3840, class_dict={'ball': 0, 'team_0': 1, 'team_1': 2}
):
    """
    Output the bounding boxes as text files in YOLOv5 Ultralytics Format.
    Inputs:
        input_df: A DataFrame of bounding boxes with corresponding images.
        output_dir: A PathLib Path which points to the directory where labels should output.
        input_type: 'top_view' or 'wide_view'
        input_height: The height of the images, as an integer in pixels.
        input_width: The width of the images, as an integer in pixels.
        class dict: A dictionary of classes to include and associated ids. Classes are strings appearing in input_df
                    and ids are an integer index starting from 0.
    Outputs:
        Saves a .txt file for every image with the path bounding boxes in YOLOv5 desired format.
    """
    if ((input_type != 'top_view') & (input_type != 'wide_view')):
        raise Exception("Please set input_type to 'top_view' or 'wide_view'.")
    
    input_df = input_df.copy().reset_index()
    # For each vidname, ensure the directory exists.
    for i in input_df.index:
        vidname, frame = input_df.loc[i, 'vidname'][0].strip(), str(input_df.loc[i, 'frame'][0]).strip()
        output_path = output_dir/"{}/{}/{}_{}.txt".format(input_type, vidname, vidname, frame)
        if not os.path.exists(output_dir/"{}/{}".format(input_type, vidname)):
            os.makedirs(output_dir/"{}/{}".format(input_type, vidname))
        # for each frame, save the relevant classes & bounding boxes to a text file.
        with open(output_path, 'w') as f:
            i = 0
            for class_name in class_dict.keys():
                class_index = str(class_dict[class_name])
                for instance in input_df[class_name].columns.get_level_values(0).unique():
                    left = input_df[class_name][instance]['bb_left'][0]
                    top = input_df[class_name][instance]['bb_top'][0]
                    width = input_df[class_name][instance]['bb_width'][0]
                    height = input_df[class_name][instance]['bb_height'][0]

                    left, width = left / input_width, width / input_width
                    top, height = top / input_height, height / input_height

                    xcenter, ycenter = left + width / 2, top + height / 2
                    
                    if i == 0:
                        to_write = " ".join((str(class_index), str(xcenter), str(ycenter), str(width), str(height)))
                    else:
                        to_write += "\n" + " ".join((str(class_index), str(xcenter), str(ycenter), str(width), str(height)))
                    i += 1
            f.write(to_write)
            f.close()


In [ ]:
format_labels_to_ultralytics(top_view_labels)

C:\Users\timot\AppData\Local\Temp\ipykernel_20180\1467538465.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vidname, frame = input_df.loc[i, 'vidname'][0].strip(), str(input_df.loc[i, 'frame'][0]).strip()
